In [1]:
try:
    passes += 1
except NameError:
    %cd ..
    passes = 1

c:\Users\susanne\git\binarybeech


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("data/titanic_kaggle.csv")
df_titanic = df.sample(frac=0.75)
df_test = df.loc[df.index.difference(df_titanic), :]

In [4]:
from binarybeech.binarybeech import *
import binarybeech.utils as utils

In [5]:
import logging

logging.basicConfig(level=logging.INFO)

In [6]:
rf_titanic = RandomForest(
    df=df_titanic.copy(),
    y_name="Survived",
    cart_settings={"max_depth": 4, "min_leaf_samples": 2, "min_split_samples": 2},
    sample_frac=0.3,
    n_attributes=5,
    method="classification",
)

In [7]:
rf_titanic.train(20)

In [8]:
rf_titanic.validate_oob()

{'precision': array([0.96314496, 0.30651341]),
 'recall': array([0.68411867, 0.84210526]),
 'F-score': 0.6247191011235955,
 'accuracy': 0.7065868263473054}

In [9]:
rf_titanic.validate()

{'precision': array([0.97542998, 0.348659  ]),
 'recall': array([0.70017637, 0.9009901 ]),
 'F-score': 0.6589787514039049,
 'accuracy': 0.7305389221556886}

In [10]:
rf_titanic.variable_importance()

{'Sex': 1.0,
 'Pclass': 0.5876288659793812,
 'Age': 0.38144329896907236,
 'PassengerId': 0.34536082474226826,
 'Fare': 0.31958762886597936,
 'SibSp': 0.19587628865979392,
 'Parch': 0.13917525773195874,
 'Embarked': 0.0979381443298969,
 'Name': 0.0,
 'Ticket': 0.0,
 'Cabin': 0.0}

In [11]:
import binarybeech.visualize as viz

viz.print_bars(rf_titanic.variable_importance(), max_width=50)

Sex       |############################### 1.0
Pclass    |##################             0.59
Age       |############                   0.38
PassengerId|###########                    0.35
Fare      |##########                     0.32
SibSp     |######                          0.2
Parch     |####                           0.14
Embarked  |###                            0.098
Name      |                                0.0
Ticket    |                                0.0
Cabin     |                                0.0


In [12]:
rf_titanic.validate(df_test)

{'precision': array([0.97814208, 0.34795322]),
 'recall': array([0.70657895, 0.90839695]),
 'F-score': 0.6618224456801749,
 'accuracy': 0.7362514029180696}

In [13]:
gbt_titanic = GradientBoostedTree(
    df=df_titanic.copy(),
    y_name="Survived",
    cart_settings={"max_depth": 2, "min_leaf_samples": 10, "min_split_samples": 10},
    sample_frac=0.75,
    n_attributes=9,
)

In [14]:
gbt_titanic.X_names

['PassengerId',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [15]:
it = gbt_titanic._initial_tree()

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [16]:
viz.print_tree(it.tree)

0.6092814371257484



In [17]:
it.tree.root

In [18]:
gbt_titanic.learning_rate = 0.01
# gbt_titanic.create_trees(200)

In [19]:
gbt_titanic.predict(df_titanic.iloc[[0]])

array([0.60928144])

In [20]:
gbt_titanic.validate(df_test)

c:\Users\susanne\git\binarybeech\binarybeech\metrics.py:34: RuntimeWarning: invalid value encountered in true_divide
  return np.diag(m) / np.sum(m, axis=0)


{'precision': array([0., 1.]),
 'recall': array([       nan, 0.38383838]),
 'F-score': nan,
 'accuracy': 0.3838383838383838}

#k-fold cross-validation

In [21]:
c = CART(df=df_titanic, y_name="Survived", method="classification")

In [22]:
c.train(slack=1e-3)

INFO:binarybeech.binarybeech:beta_best: 0.0035785287704622527


In [23]:
c.tree.leaf_count()

6

In [24]:
viz.print_tree(c.tree)

Sex in ('female',)
├── False:0
└── True: Pclass<3.00
    ├── False:Fare<24.54
    │   ├── False:0
    │   └── True: Embarked in ('S',)
    │       ├── False:1
    │       └── True: PassengerId<626.41
    │           ├── False:1
    │           └── True: 0
    └── True: 1



In [25]:
c.validate(df_test)

{'precision': array([0.9435337 , 0.60526316]),
 'recall': array([0.79326187, 0.8697479 ]),
 'F-score': 0.7878449710253026,
 'accuracy': 0.813692480359147}